# Setup

In [1]:
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import blempamhmuuuuummm as bph


# for one game. Used for testing
# test_game_path = "test_game.slp"
# bph.write_game_json(test_game_path)
# load_single_game('game_analytics/test_game.json', pd.DataFrame())

# write game json files
slippi_archive_path = "../../Slippi"
bph.write_games_json(slippi_archive_path)

# read game json into memory
all_games = bph.load_multiple_games("game_analytics")
my_id = bph.find_my_id(all_games) # this guesses your id based on the most popular id in your archive. 
# my_id = "aPmT3TcxYLaUAlxSDKDsfKFmXC72" # or you can set it manually


something went wrong with:  game_analytics/Game_20231002T112717.json
something went wrong with:  game_analytics/Game_20230821T074359.json
something went wrong with:  game_analytics/.DS_Store
something went wrong with:  game_analytics/Game_20231003T144755.json
something went wrong with:  game_analytics/Game_20230810T141105.json
something went wrong with:  game_analytics/Game_20231001T235749.json
something went wrong with:  game_analytics/Game_20231018T195855.json
something went wrong with:  game_analytics/Game_20230827T101530.json
something went wrong with:  game_analytics/Game_20230807T091818.json
something went wrong with:  game_analytics/Game_20230806T200651.json
something went wrong with:  game_analytics/Game_20230814T213419.json
something went wrong with:  game_analytics/Game_20230722T085734.json
something went wrong with:  game_analytics/Game_20231021T081930.json
something went wrong with:  game_analytics/Game_20230906T150537.json
something went wrong with:  game_analytics/Game_20

# Sample Queries

In [3]:
#scoping
completed_games = bph.finished_games(all_games)
recent = bph.recent_games(completed_games, 50) # recent n number of games

In [4]:
# win rates
bph.overall_win_rates(completed_games, my_id)

(2399, 2741, 0.46673151750972763)

In [5]:
bph.win_rates_for_all_stages(completed_games, my_id)

{'Dream Land N64': 0.5213675213675214,
 'Final Destination': 0.4827586206896552,
 "Yoshi's Story": 0.4734982332155477,
 'Pokémon Stadium': 0.4641638225255973,
 'Fountain of Dreams': 0.451348182883939,
 'Battlefield': 0.4310747663551402,
 "Princess Peach's Castle": 0,
 'Kongo Jungle': 0,
 'Brinstar': 0,
 'Corneria': 0,
 'Onett': 0,
 'Mute City': 0,
 'Rainbow Cruise': 0,
 'Jungle Japes': 0,
 'Great Bay': 0,
 'Hyrule Temple': 0,
 'Brinstar Depths': 0,
 "Yoshi's Island": 0,
 'Green Greens': 0,
 'Fourside': 0,
 'Mushroom Kingdom I': 0,
 'Mushroom Kingdom IIVenom': 0,
 'Poké Floats': 0,
 'Big Blue': 0,
 'Icicle Mountain': 0,
 'Icetop': 0,
 'Flat Zone': 0,
 "Yoshi's Island N64": 0,
 'Kongo Jungle N64': 0}

In [6]:
bph.encounter_rates_across_characters(all_games, my_id)

{'Falco': 0.2056420233463035,
 'Fox': 0.15369649805447472,
 'Captain Falcon': 0.1208171206225681,
 'Marth': 0.12062256809338522,
 'Sheik': 0.06750972762645914,
 'Jigglypuff': 0.03832684824902724,
 'Dr. Mario': 0.03151750972762646,
 'Yoshi': 0.030155642023346304,
 'Luigi': 0.02821011673151751,
 'Ganondorf': 0.027431906614785992,
 'Peach': 0.022957198443579768,
 'Samus': 0.020622568093385214,
 'Mario': 0.017120622568093387,
 'Ice Climbers': 0.015953307392996108,
 'Pikachu': 0.01536964980544747,
 'Donkey Kong': 0.014785992217898832,
 'Roy': 0.014007782101167316,
 'Young Link': 0.011673151750972763,
 'Link': 0.011284046692607004,
 'Mewtwo': 0.0071984435797665365,
 'Mr. Game & Watch': 0.007003891050583658,
 'Bowser': 0.004280155642023347,
 'Ness': 0.004085603112840467,
 'Kirby': 0.003501945525291829,
 'Pichu': 0.002723735408560311,
 'Zelda': 0.002529182879377432}

In [8]:
bph.l_cancel_rate(completed_games, my_id, False)

0.5443554606394969

In [9]:
bph.get_character_win_rates(completed_games, my_id)

{'Sheik': 0.3032069970845481,
 'Pikachu': 0.379746835443038,
 'Zelda': 0.38461538461538464,
 'Captain Falcon': 0.3883495145631068,
 'Peach': 0.4152542372881356,
 'Samus': 0.42452830188679247,
 'Ganondorf': 0.4420289855072464,
 'Mr. Game & Watch': 0.4444444444444444,
 'Yoshi': 0.45806451612903226,
 'Fox': 0.4648786717752235,
 'Falco': 0.4661582459485224,
 'Marth': 0.4773462783171521,
 'Link': 0.4827586206896552,
 'Dr. Mario': 0.4876543209876543,
 'Luigi': 0.4896551724137931,
 'Donkey Kong': 0.6052631578947368,
 'Ice Climbers': 0.6075949367088608,
 'Ness': 0.6190476190476191,
 'Bowser': 0.6363636363636364,
 'Roy': 0.6388888888888888,
 'Jigglypuff': 0.649746192893401,
 'Mario': 0.6704545454545454,
 'Young Link': 0.6833333333333333,
 'Kirby': 0.7777777777777778,
 'Pichu': 0.7857142857142857,
 'Mewtwo': 0.8378378378378378}

In [10]:
bph.all_character_tech_options(completed_games, my_id)

{'Captain Falcon': tech_away       0.061726
 tech_in         0.053247
 tech_fail       0.654061
 tech_neutral    0.230965
 dtype: float64,
 'Donkey Kong': tech_away       0.050292
 tech_in         0.078363
 tech_fail       0.651462
 tech_neutral    0.219883
 dtype: float64,
 'Fox': tech_away       0.043385
 tech_in         0.047022
 tech_fail       0.708165
 tech_neutral    0.201428
 dtype: float64,
 'Mr. Game & Watch': tech_away       0.028571
 tech_in         0.020408
 tech_fail       0.763265
 tech_neutral    0.187755
 dtype: float64,
 'Kirby': tech_away       0.0375
 tech_in         0.0875
 tech_fail       0.7125
 tech_neutral    0.1625
 dtype: float64,
 'Bowser': tech_away       0.019231
 tech_in         0.070513
 tech_fail       0.685897
 tech_neutral    0.224359
 dtype: float64,
 'Link': tech_away       0.036244
 tech_in         0.054366
 tech_fail       0.742998
 tech_neutral    0.166392
 dtype: float64,
 'Luigi': tech_away       0.032821
 tech_in         0.072821
 tech_fail   

In [6]:
bph.get_opponent_history(all_games, "rat")

ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [12]:
bph.tech_options(all_games, my_id, False)

(43950,
 tech_away        1968
 tech_in          2638
 tech_fail       30419
 tech_neutral     8925
 dtype: int64,
 tech_away       0.044778
 tech_in         0.060023
 tech_fail       0.692127
 tech_neutral    0.203072
 dtype: float64)

In [17]:
bph.avg_shmove_counts(completed_games, my_id)

wavedash_count      0.222374
waveland_count      0.236965
airdodge_count      1.566537
dashdance_count     1.573152
spotdodge_count     0.439300
roll_count         26.909144
ledgegrab_count     4.141051
dtype: float64

In [18]:
bph.avg_shmove_counts(completed_games, my_id, False)

wavedash_count     16.692996
waveland_count      4.058755
airdodge_count      3.131907
dashdance_count    23.658755
spotdodge_count     1.342412
roll_count          5.626459
ledgegrab_count     4.613813
dtype: float64